In [ ]:
import sys
import random
import hashlib

sys.path.append('../')
import util
from test_framework.key import SECP256K1_ORDER, SECP256K1_FIELD_SIZE
from test_framework.key import ECKey, ECPubKey, jacobi_symbol

# 1.1 Introduction to Schnorr

* Part 1: Schnorr Signatures.
* Part 2: Deterministic Nonces.

## Part 1: Schnorr Signatures

The Schnorr signature equation is the following:

* `S = R + H(x(R)|P|msg) * P`

Signing involves generating a secret nonce first.

* Generate secret scalar `k`

Then computing s from:

* `s = k - H(x(R)|P|m)`

The resulting signature is:

* `x(R), s`



![test](../images/schnorr0.jpg)



### Constraint on the private nonce k.

The Schnorr signature provides the verifier with the x-coordinate of the Nonce point R and s value. For a given x-coordinate on the SECP251K1 curve, there are two possible curve points:

* `y^2 = x^3 + 7`
    * For x, both P(x,y) and P(x,-y) are valid curve points.
    * One of the y-coordinates is even, and the other one is odd.

The nonce k is therefore constrained by requiring it to be a quadratic residue modulo the field order p (SECP256k1 curve field size). 

* `k` and `SECP26k1_order - k` have nonce points `R(x, y)` and `R(x, -y)` respectively.
* Only one will be a quadratic residue modulo the field order.
    
Whether a scalar is a quadratic residue modulo the secp256k1 field order is determined with the jacobi symbol:

* Bitcoin Core method: `jacobi_symbol(y(R), SECP256K1_FIELD_SIZE) == 1`
* If the jacobisymbol is not 1, simply take the inverse of k: `k' = SECP26k1_order - k`


#### 1.1 _Programming Exercise:_ Sign a message with Schnorr

* Sign the message with the provided key pair below.

In [ ]:
msg = hashlib.sha256(b'message').digest()
x = ECKey()
x.generate()
P = x.get_pubkey()

# We have to set uncompressed to get the y-coordinate.
# k_int = random.randrange(1, SECP256K1_ORDER)
k = ECKey()
k.generate(False) # Uncompressed.
R = k.get_pubkey()

# Check that nonce is quadratic residue modulo the field order.
# Method: jacobi_symbol(int(y(R)), SECP256K1_FIELD_SIZE)
if jacobi_symbol(int.from_bytes(R.get_bytes()[33:], "big"), SECP256K1_FIELD_SIZE) != 1:
    k.negate()

# Generate s = k + sha256(R_x|P|msg) * x
# Method: hashlib.sha256(bytes)
h_b = hashlib.sha256(R.get_bytes()[1:33] + P.get_bytes() + msg).digest()
h = ECKey()
h.set(h_b, True)
s = k + h * x

# Generate sig = R_x|s
sig = R.get_bytes()[1:33] + s.secret.to_bytes(32,'big')

# Verify the signature
# Method: ECPubKey.verify_schnorr(sig, msg)
print("Signature verification:", P.verify_schnorr(sig, msg))


## Part 2: Deterministic Nonces for Schnorr signatures

So far we have used a random secret nonce for creating Schnorr signatures. This has the disadvantage that the the user must rely on the robustness of the random generator for each signing rounds. If the nonce generator is biased or even compromised, the private key can be derived for a given signature and known nonce.

Therefore, for the single signer Schnorr signatures, the [Schnorr BIP](https://github.com/sipa/bips/blob/bip-schnorr/bip-schnorr.mediawiki) proposes a deterministic nonce generation scheme.

* `k = sha256(x|msg)`

#### 2.1 _Programming Exercise:_ Signing Schnorr with deterministic nonce.

* Create a Schnorr signature with the deterministic nonce scheme proposed in the Schnorr BIP.
* Compare this signature to the private key class method `ECKey.sign_schnorr(msg)`

In [ ]:
msg = hashlib.sha256(b'message').digest()
x = ECKey()
x.generate()
P = x.get_pubkey()

# We have to set uncompressed to get the y-coordinate.
k_b = hashlib.sha256(x.get_bytes() + msg).digest()
k_int = int.from_bytes(k_b, "big")
k = ECKey()
k.set(k_b, False) 
R = k.get_pubkey()

# Check that nonce is quadratic residue modulo the field order.
if jacobi_symbol(int.from_bytes(R.get_bytes()[33:], "big"), SECP256K1_FIELD_SIZE) != 1:
    k.negate()

# Generate s = k + H(R_x|P|msg) * x
h_b = hashlib.sha256(R.get_bytes()[1:33] + P.get_bytes() + msg).digest()
h = ECKey()
h.set(h_b, True)
s = k + h * x

# Generate sig = R_x|s
# Also, generate alternative sig with ECKey.sign_schnorr(msg)
sig = R.get_bytes()[1:33] + s.secret.to_bytes(32,'big')
sig2 = x.sign_schnorr(msg)

# Verify and compare signature(s). 
print("Sig1 verification:", P.verify_schnorr(sig, msg))
print("Sig2 verification:", P.verify_schnorr(sig2, msg))
print("Signature equality:", sig == sig2)
